In [2]:
!pip install opencv-python


In [9]:
from main_test_swinir import main
import argparse

args = argparse.Namespace(
    task='real_sr',
    scale=4,
    noise=15,
    jpeg=40,
    training_patch_size=128,
    large_model=True,
    model_path='model_zoo/swinir/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth',
    folder_lq='testsets/RealSRSet+5images',
    folder_gt=None,
    tile=None,
    tile_overlap=32
)

main(args)


TypeError: main() takes 0 positional arguments but 1 was given

In [4]:
!pip install timm

     ---------------------------------------- 0.0/52.0 kB ? eta -:--:--
     -------------- ----------------------- 20.5/52.0 kB 330.3 kB/s eta 0:00:01
     -------------------------------------  51.2/52.0 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 52.0/52.0 kB 383.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.4 MB 1.7 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/2.4 MB 3.8 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.4 MB 5.5 MB/s eta 0:00:01
   ------------ --------------------------- 0.7/2.4 MB 5.0 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.4 MB 5.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.7/2.4 MB 6.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.0/2.4 MB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 6.5 MB/s eta 0:00:00
   --------